# Open AI मॉडल्स का फाइन ट्यूनिंग

यह नोटबुक Open AI के [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) दस्तावेज़ में दी गई वर्तमान मार्गदर्शन पर आधारित है।

फाइन-ट्यूनिंग आपके एप्लिकेशन के लिए फाउंडेशन मॉडल्स के प्रदर्शन को बेहतर बनाता है, इसे उस विशिष्ट उपयोग मामले या परिदृश्य से संबंधित अतिरिक्त डेटा और संदर्भ के साथ पुनः प्रशिक्षित करके। ध्यान दें कि प्रॉम्प्ट इंजीनियरिंग तकनीकें जैसे _few shot learning_ और _retrieval augmented generation_ आपको गुणवत्ता सुधारने के लिए प्रासंगिक डेटा के साथ डिफ़ॉल्ट प्रॉम्प्ट को बढ़ाने की अनुमति देती हैं। हालांकि, ये दृष्टिकोण लक्षित फाउंडेशन मॉडल की अधिकतम टोकन विंडो आकार द्वारा सीमित हैं।

फाइन-ट्यूनिंग के साथ, हम प्रभावी रूप से आवश्यक डेटा के साथ मॉडल को पुनः प्रशिक्षित कर रहे हैं (जो हमें अधिकतम टोकन विंडो में फिट होने वाले कई उदाहरणों का उपयोग करने की अनुमति देता है) - और मॉडल का एक _कस्टम_ संस्करण तैनात कर रहे हैं जिसे अब अनुमान लगाने के समय उदाहरण प्रदान करने की आवश्यकता नहीं होती। यह न केवल हमारे प्रॉम्प्ट डिज़ाइन की प्रभावशीलता को बढ़ाता है (हम टोकन विंडो का उपयोग अन्य चीजों के लिए अधिक लचीलापन के साथ कर सकते हैं) बल्कि संभावित रूप से हमारे लागतों में भी सुधार करता है (अनुमान लगाने के समय मॉडल को भेजे जाने वाले टोकन की संख्या को कम करके)।

फाइन ट्यूनिंग के 4 चरण हैं:
1. प्रशिक्षण डेटा तैयार करें और उसे अपलोड करें।
1. फाइन-ट्यून मॉडल प्राप्त करने के लिए प्रशिक्षण कार्य चलाएं।
1. फाइन-ट्यून मॉडल का मूल्यांकन करें और गुणवत्ता के लिए पुनरावृत्ति करें।
1. संतुष्ट होने पर अनुमान लगाने के लिए फाइन-ट्यून मॉडल को तैनात करें।

ध्यान दें कि सभी फाउंडेशन मॉडल फाइन-ट्यूनिंग का समर्थन नहीं करते - नवीनतम जानकारी के लिए [OpenAI दस्तावेज़](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) देखें। आप पहले से फाइन-ट्यून किए गए मॉडल को भी फाइन-ट्यून कर सकते हैं। इस ट्यूटोरियल में, हम फाइन-ट्यूनिंग के लिए अपने लक्षित फाउंडेशन मॉडल के रूप में `gpt-35-turbo` का उपयोग करेंगे।

---


### चरण 1.1: अपना डेटासेट तैयार करें

आइए एक चैटबॉट बनाएं जो आपको तत्वों की आवर्त सारणी को समझने में मदद करे, एक तत्व के बारे में प्रश्नों का उत्तर एक लिमरिक के साथ देकर। इस _सरल_ ट्यूटोरियल में, हम केवल मॉडल को प्रशिक्षित करने के लिए कुछ नमूना उत्तरों के साथ एक डेटासेट बनाएंगे जो डेटा के अपेक्षित प्रारूप को दिखाते हैं। वास्तविक दुनिया के उपयोग के मामले में, आपको बहुत अधिक उदाहरणों के साथ एक डेटासेट बनाना होगा। आप एक खुला डेटासेट (अपने आवेदन डोमेन के लिए) भी उपयोग कर सकते हैं यदि ऐसा कोई मौजूद हो, और उसे फाइन-ट्यूनिंग के लिए पुन: स्वरूपित कर सकते हैं।

चूंकि हम `gpt-35-turbo` पर ध्यान केंद्रित कर रहे हैं और एकल-टर्न प्रतिक्रिया (चैट पूर्णता) की तलाश में हैं, हम [इस सुझाए गए प्रारूप](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) का उपयोग करके उदाहरण बना सकते हैं जो OpenAI चैट पूर्णता आवश्यकताओं को दर्शाता है। यदि आप बहु-टर्न संवादात्मक सामग्री की उम्मीद करते हैं, तो आप [बहु-टर्न उदाहरण प्रारूप](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) का उपयोग करेंगे जिसमें एक `weight` पैरामीटर शामिल होता है जो संकेत देता है कि कौन से संदेश फाइन-ट्यूनिंग प्रक्रिया में उपयोग किए जाने चाहिए (या नहीं)।

हम यहां अपने ट्यूटोरियल के लिए सरल एकल-टर्न प्रारूप का उपयोग करेंगे। डेटा [jsonl प्रारूप](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) में है जिसमें प्रति पंक्ति 1 रिकॉर्ड होता है, प्रत्येक JSON-स्वरूपित ऑब्जेक्ट के रूप में प्रस्तुत। नीचे दिया गया स्निपेट 2 रिकॉर्ड को एक नमूना के रूप में दिखाता है - पूर्ण नमूना सेट (10 उदाहरण) के लिए [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) देखें जिसका हम अपने फाइन-ट्यूनिंग ट्यूटोरियल में उपयोग करेंगे। **ध्यान दें:** प्रत्येक रिकॉर्ड _अवश्य_ एक ही पंक्ति में परिभाषित होना चाहिए (जैसे कि एक स्वरूपित JSON फ़ाइल में आमतौर पर कई पंक्तियों में विभाजित नहीं किया जाता है)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

वास्तविक दुनिया के उपयोग के मामले में अच्छे परिणामों के लिए आपको बहुत बड़ा उदाहरण सेट चाहिए होगा - समझौता उत्तरों की गुणवत्ता और फाइन-ट्यूनिंग के समय/लागत के बीच होगा। हम एक छोटा सेट उपयोग कर रहे हैं ताकि हम प्रक्रिया को समझाने के लिए फाइन-ट्यूनिंग जल्दी पूरी कर सकें। अधिक जटिल फाइन-ट्यूनिंग ट्यूटोरियल के लिए [इस OpenAI कुकबुक उदाहरण](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) को देखें।


---

### चरण 1.2 अपना डेटासेट अपलोड करें

डेटा को Files API का उपयोग करके [यहाँ वर्णित अनुसार](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file) अपलोड करें। ध्यान दें कि इस कोड को चलाने के लिए, आपको पहले निम्नलिखित चरण करने होंगे:
 - `openai` Python पैकेज इंस्टॉल किया होना चाहिए (सुनिश्चित करें कि आप नवीनतम सुविधाओं के लिए संस्करण >=0.28.0 का उपयोग कर रहे हैं)
 - `OPENAI_API_KEY` पर्यावरण चर को अपने OpenAI API कुंजी पर सेट किया होना चाहिए
अधिक जानने के लिए, कोर्स के लिए प्रदान किए गए [सेटअप गाइड](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) को देखें।

अब, अपने स्थानीय JSONL फ़ाइल से अपलोड के लिए एक फ़ाइल बनाने के लिए कोड चलाएँ।


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### चरण 2.1: SDK के साथ फाइन-ट्यूनिंग जॉब बनाएं


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### चरण 2.2: जॉब की स्थिति जांचें

यहाँ कुछ चीजें हैं जो आप `client.fine_tuning.jobs` API के साथ कर सकते हैं:
- `client.fine_tuning.jobs.list(limit=<n>)` - अंतिम n फाइन-ट्यूनिंग जॉब्स की सूची बनाएं
- `client.fine_tuning.jobs.retrieve(<job_id>)` - किसी विशिष्ट फाइन-ट्यूनिंग जॉब का विवरण प्राप्त करें
- `client.fine_tuning.jobs.cancel(<job_id>)` - किसी फाइन-ट्यूनिंग जॉब को रद्द करें
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - जॉब से n तक की घटनाओं की सूची बनाएं
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

प्रक्रिया का पहला चरण _ट्रेनिंग फ़ाइल को मान्य करना_ है ताकि यह सुनिश्चित किया जा सके कि डेटा सही प्रारूप में है।


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### चरण 2.3: प्रगति की निगरानी के लिए घटनाओं को ट्रैक करें


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### चरण 2.4: OpenAI डैशबोर्ड में स्थिति देखें


आप OpenAI वेबसाइट पर जाकर और प्लेटफ़ॉर्म के _Fine-tuning_ सेक्शन का अन्वेषण करके भी स्थिति देख सकते हैं। यह आपको वर्तमान जॉब की स्थिति दिखाएगा, और साथ ही पिछले जॉब निष्पादन रन के इतिहास को ट्रैक करने देगा। इस स्क्रीनशॉट में, आप देख सकते हैं कि पिछला निष्पादन विफल रहा, और दूसरा रन सफल हुआ। संदर्भ के लिए, यह तब हुआ जब पहले रन ने JSON फ़ाइल का उपयोग किया जिसमें रिकॉर्ड गलत तरीके से स्वरूपित थे - एक बार ठीक होने के बाद, दूसरा रन सफलतापूर्वक पूरा हुआ और मॉडल को उपयोग के लिए उपलब्ध करवा दिया। 

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.hi.png)


आप नीचे स्क्रॉल करके विज़ुअल डैशबोर्ड में स्थिति संदेश और मेट्रिक्स भी देख सकते हैं जैसा कि दिखाया गया है:

| Messages | Metrics |
|:---|:---|
| ![Messages](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.hi.png) |  ![Metrics](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.hi.png)|


---

### चरण 3.1: आईडी प्राप्त करें और कोड में फाइन-ट्यून मॉडल का परीक्षण करें


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### चरण 3.2: प्लेग्राउंड में फाइन-ट्यून मॉडल लोड और परीक्षण करें

अब आप फाइन-ट्यून मॉडल को दो तरीकों से परीक्षण कर सकते हैं। पहला, आप प्लेग्राउंड पर जा सकते हैं और मॉडल ड्रॉप-डाउन का उपयोग करके सूचीबद्ध विकल्पों में से अपने नए फाइन-ट्यून मॉडल का चयन कर सकते हैं। दूसरा विकल्प फाइन-ट्यूनिंग पैनल में दिखाए गए "प्लेग्राउंड" विकल्प का उपयोग करना है (ऊपर स्क्रीनशॉट देखें) जो निम्नलिखित _तुलनात्मक_ दृश्य लॉन्च करता है जो फाउंडेशन और फाइन-ट्यून मॉडल संस्करणों को एक साथ त्वरित मूल्यांकन के लिए दिखाता है।

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.hi.png)

बस अपने प्रशिक्षण डेटा में उपयोग किए गए सिस्टम संदर्भ को भरें और अपना परीक्षण प्रश्न प्रदान करें। आप देखेंगे कि दोनों पक्ष समान संदर्भ और प्रश्न के साथ अपडेट होते हैं। तुलना चलाएं और आप उनके आउटपुट में अंतर देखेंगे। _ध्यान दें कि फाइन-ट्यून मॉडल आपके उदाहरणों में प्रदान किए गए प्रारूप में प्रतिक्रिया प्रस्तुत करता है जबकि फाउंडेशन मॉडल केवल सिस्टम प्रॉम्प्ट का पालन करता है_।

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.hi.png)

आप देखेंगे कि तुलना प्रत्येक मॉडल के लिए टोकन गणना और अनुमानित समय भी प्रदान करती है। **यह विशिष्ट उदाहरण एक सरल प्रक्रिया दिखाने के लिए है लेकिन वास्तव में किसी वास्तविक दुनिया के डेटा सेट या परिदृश्य को प्रतिबिंबित नहीं करता है**। आप देख सकते हैं कि दोनों नमूने समान संख्या में टोकन दिखाते हैं (सिस्टम संदर्भ और उपयोगकर्ता प्रॉम्प्ट समान हैं) जबकि फाइन-ट्यून मॉडल अनुमान के लिए अधिक समय लेता है (कस्टम मॉडल)।

वास्तविक दुनिया के परिदृश्यों में, आप इस तरह के खिलौना उदाहरण का उपयोग नहीं करेंगे, बल्कि वास्तविक डेटा (जैसे ग्राहक सेवा के लिए उत्पाद कैटलॉग) के खिलाफ फाइन-ट्यूनिंग करेंगे जहां प्रतिक्रिया की गुणवत्ता बहुत अधिक स्पष्ट होगी। _उस_ संदर्भ में, फाउंडेशन मॉडल के साथ समान प्रतिक्रिया गुणवत्ता प्राप्त करने के लिए अधिक कस्टम प्रॉम्प्ट इंजीनियरिंग की आवश्यकता होगी जो टोकन उपयोग बढ़ाएगा और संभावित रूप से अनुमान के लिए संबंधित प्रसंस्करण समय भी बढ़ाएगा। _इसे आजमाने के लिए, OpenAI कुकबुक में फाइन-ट्यूनिंग उदाहरण देखें और शुरुआत करें।_

---


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता के लिए प्रयासरत हैं, कृपया ध्यान दें कि स्वचालित अनुवादों में त्रुटियाँ या अशुद्धियाँ हो सकती हैं। मूल दस्तावेज़ अपनी मूल भाषा में ही अधिकारिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सलाह दी जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम जिम्मेदार नहीं हैं।
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
